In [24]:
# a.

import pandas as pd
import numpy as np

df = pd.read_csv('/content/accidentsFull.csv', index_col=None)

df['INJURY'] = np.where(df['MAX_SEV_IR'] > 0, 1, 0)

num_injuries = sum(df['INJURY']==1)

print(f"{num_injuries/len(df)=:.4f}")

num_injuries/len(df)=0.5088


In [25]:
df.columns

Index(['HOUR_I_R', 'ALCHL_I', 'ALIGN_I', 'STRATUM_R', 'WRK_ZONE', 'WKDY_I_R',
       'INT_HWY', 'LGTCON_I_R', 'MANCOL_I_R', 'PED_ACC_R', 'RELJCT_I_R',
       'REL_RWY_R', 'PROFIL_I_R', 'SPD_LIM', 'SUR_COND', 'TRAF_CON_R',
       'TRAF_WAY', 'VEH_INVL', 'WEATHER_R', 'INJURY_CRASH', 'NO_INJ_I',
       'PRPTYDMG_CRASH', 'FATALITIES', 'MAX_SEV_IR', 'INJURY'],
      dtype='object')

In [26]:
# b.

df_b = df[['WEATHER_R', 'TRAF_CON_R', 'INJURY']][:12]
print(df_b)

    WEATHER_R  TRAF_CON_R  INJURY
0           1           0       1
1           2           0       0
2           2           1       0
3           1           1       0
4           1           0       0
5           2           0       1
6           2           0       0
7           1           0       1
8           2           0       0
9           2           0       0
10          2           0       0
11          1           2       0


In [4]:
# b.i

pivot_table = df_b.pivot_table(
    values='INJURY',
    index='WEATHER_R',
    columns='TRAF_CON_R')

print(pivot_table)

TRAF_CON_R         0    1    2
WEATHER_R                     
1           0.666667  0.0  0.0
2           0.166667  0.0  NaN


In [54]:
# b.v

exact_probs = [2/3, 1/6, 0, 0, 2/3, 1/6, 1/6, 2/3, 1/6, 1/6, 1/6, 0]

from sklearn.naive_bayes import CategoricalNB

X = df_b.drop('INJURY', axis=1)
y = df_b['INJURY']

nb_classifier = CategoricalNB(alpha=0)
nb_classifier.fit(X, y)

y_pred = nb_classifier.predict(X)
y_prob = nb_classifier.predict_proba(X)

for idx in range(len(y_pred)):
    print(f"data {idx:2d}. 정답: {y[idx]}")
    print(f"{'Exact':5} | 예측: {int(exact_probs[idx]>=0.5)}, 부상 확률: {exact_probs[idx]:.4f}")
    print(f"{'Naive':5} | 예측: {y_pred[idx]}, 부상 확률: {y_prob[idx][1]:.4f}")

data  0. 정답: 1
Exact | 예측: 1, 부상 확률: 0.6667
Naive | 예측: 1, 부상 확률: 0.5000
data  1. 정답: 0
Exact | 예측: 0, 부상 확률: 0.1667
Naive | 예측: 0, 부상 확률: 0.2000
data  2. 정답: 0
Exact | 예측: 0, 부상 확률: 0.0000
Naive | 예측: 0, 부상 확률: 0.0000
data  3. 정답: 0
Exact | 예측: 0, 부상 확률: 0.0000
Naive | 예측: 0, 부상 확률: 0.0000
data  4. 정답: 0
Exact | 예측: 1, 부상 확률: 0.6667
Naive | 예측: 1, 부상 확률: 0.5000
data  5. 정답: 1
Exact | 예측: 0, 부상 확률: 0.1667
Naive | 예측: 0, 부상 확률: 0.2000
data  6. 정답: 0
Exact | 예측: 0, 부상 확률: 0.1667
Naive | 예측: 0, 부상 확률: 0.2000
data  7. 정답: 1
Exact | 예측: 1, 부상 확률: 0.6667
Naive | 예측: 1, 부상 확률: 0.5000
data  8. 정답: 0
Exact | 예측: 0, 부상 확률: 0.1667
Naive | 예측: 0, 부상 확률: 0.2000
data  9. 정답: 0
Exact | 예측: 0, 부상 확률: 0.1667
Naive | 예측: 0, 부상 확률: 0.2000
data 10. 정답: 0
Exact | 예측: 0, 부상 확률: 0.1667
Naive | 예측: 0, 부상 확률: 0.2000
data 11. 정답: 0
Exact | 예측: 0, 부상 확률: 0.0000
Naive | 예측: 0, 부상 확률: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:1504: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cat_count) - np.log(smoothed_class_count.reshape(-1, 1))


In [6]:
# c.i

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix


df_c = df[['HOUR_I_R', 'WKDY_I_R', 'LGTCON_I_R',
           'ALIGN_I', 'WRK_ZONE', 'INT_HWY', 'RELJCT_I_R', 'REL_RWY_R', 'PROFIL_I_R', 'SPD_LIM', 'TRAF_CON_R', 'TRAF_WAY',
           'SUR_COND', 'WEATHER_R',
           'INJURY']]

In [55]:
# c.ii & c.iii

X = df_c.drop(['INJURY'], axis=1)
y = df_c['INJURY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

nb_classifier = CategoricalNB(alpha=0)
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)
y_prob = nb_classifier.predict_proba(X_test)
acc_sco = accuracy_score(y_test, y_pred)
conf_mat = confusion_matrix(y_test, y_pred)
error_rate = 1 - acc_sco

print("confusion matrix:\n", conf_mat)
print(f"error rate: {error_rate:.4f}")


confusion matrix:
 [[3783 4490]
 [3391 5210]]
error rate: 0.4670


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:1504: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cat_count) - np.log(smoothed_class_count.reshape(-1, 1))


In [57]:
# c.v

pivot_table = df_c.pivot_table(
    values='INJURY',
    index='SPD_LIM',
    )

print(pivot_table)

           INJURY
SPD_LIM          
5        0.666667
10       0.500000
15       0.491803
20       0.366534
25       0.466112
30       0.513594
35       0.532373
40       0.540428
45       0.508122
50       0.493093
55       0.498635
60       0.561520
65       0.495028
70       0.437414
75       0.554770


In [56]:
print(df_c[df_c['SPD_LIM']==5])

       HOUR_I_R  WKDY_I_R  LGTCON_I_R  ALIGN_I  WRK_ZONE  INT_HWY  RELJCT_I_R  \
9827          0         0           2        1         0        0           0   
11278         0         0           2        1         0        0           1   
11847         0         1           2        1         0        0           1   
24185         0         0           1        2         0        0           1   
33330         1         1           1        1         0        0           1   
40366         0         1           1        1         0        0           1   

       REL_RWY_R  PROFIL_I_R  SPD_LIM  TRAF_CON_R  TRAF_WAY  SUR_COND  \
9827           0           1        5           0         1         1   
11278          1           0        5           0         1         1   
11847          1           0        5           2         1         1   
24185          1           0        5           2         3         1   
33330          1           0        5           1         1        